## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-27-02-10-01 +0000,nyt,Des Moines Schools Superintendent Ian Roberts ...,https://www.nytimes.com/2025/09/26/us/des-moin...
1,2025-09-27-02-05-05 +0000,nyt,Why Companies Like Disney and Paramount Are Ca...,https://www.nytimes.com/2025/09/26/business/tr...
2,2025-09-27-02-03-00 +0000,wsj,The Supreme Court on Friday granted the Trump ...,https://www.wsj.com/politics/policy/supreme-co...
3,2025-09-27-02-02-51 +0000,nyt,Justice Dept. Agents to Protect ICE Facilities...,https://www.nytimes.com/2025/09/26/us/doj-agen...
4,2025-09-27-02-00-00 +0000,wsj,A U.S. plan to field thousands of cutting-edge...,https://www.wsj.com/politics/national-security...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2398/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,62
108,comey,24
52,new,23
415,will,16
378,tariffs,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
149,2025-09-26-18-49-37 +0000,nypost,Trump predicts ‘there will be others’ after Ja...,https://nypost.com/2025/09/26/us-news/trump-pr...,136
113,2025-09-26-20-38-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...,123
20,2025-09-27-01-00-00 +0000,wsj,The indictment of James Comey sealed the remar...,https://www.wsj.com/politics/policy/trump-over...,123
190,2025-09-26-16-22-19 +0000,bbc,Watch: Trump says 'there will be others' after...,https://www.bbc.com/news/videos/ceq2zzp1dx2o?a...,120
253,2025-09-26-11-55-31 +0000,nypost,Trump rages about ‘dirty cop’ Comey’s case bei...,https://nypost.com/2025/09/26/us-news/trump-ra...,117


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
149,136,2025-09-26-18-49-37 +0000,nypost,Trump predicts ‘there will be others’ after Ja...,https://nypost.com/2025/09/26/us-news/trump-pr...
189,78,2025-09-26-16-22-23 +0000,nypost,"Netanyahu shrugs off UN walkout, says Israel m...",https://nypost.com/2025/09/26/us-news/netanyah...
4,70,2025-09-27-02-00-00 +0000,wsj,A U.S. plan to field thousands of cutting-edge...,https://www.wsj.com/politics/national-security...
2,50,2025-09-27-02-03-00 +0000,wsj,The Supreme Court on Friday granted the Trump ...,https://www.wsj.com/politics/policy/supreme-co...
109,49,2025-09-26-20-43-00 +0000,wsj,Even as Senate Minority Leader Chuck Schumer s...,https://www.wsj.com/politics/policy/government...
31,43,2025-09-27-00-13-43 +0000,nypost,NYC woman who sucker-punched pro-life activist...,https://nypost.com/2025/09/26/us-news/manhatta...
264,40,2025-09-26-10-32-37 +0000,nypost,Iran and Russia sign $25 billion agreement to ...,https://nypost.com/2025/09/26/world-news/iran-...
65,37,2025-09-26-22-28-20 +0000,nypost,Trump backs plan to make ex-UK PM Tony Blair ‘...,https://nypost.com/2025/09/26/world-news/trump...
89,36,2025-09-26-21-25-31 +0000,nyt,Sinclair and Nexstar End Jimmy Kimmel Boycott ...,https://www.nytimes.com/2025/09/26/business/me...
113,36,2025-09-26-20-38-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
